# Imputación con Random forest

## Preparar librerias

In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import random
import warnings
import math
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import importlib.util

# Especifica la ruta al archivo del módulo
ruta_al_modulo = './util/auxi.py'

# Importa el módulo
spec = importlib.util.spec_from_file_location("auxi", ruta_al_modulo)
auxi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(auxi)

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [240]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
serie_temperatura = dataframe_temperatura.squeeze('columns')
serie_temperatura = serie_temperatura.resample('D').mean()
serie_temperatura['1984-12-31'] = (serie_temperatura['1984-12-30'] + serie_temperatura['1985-01-01'])/2
serie_temperatura['1988-12-31'] = (serie_temperatura['1988-12-30'] + serie_temperatura['1989-01-01'])/2
serie_temperatura.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [241]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
serie_cardiovascular = dataframe_cardiovascular.squeeze('columns')
serie_cardiovascular = serie_cardiovascular.astype(float)
serie_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_3300\3747356855.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136.0
2017-08-03 01:00:00    46450.0
2017-08-03 02:00:00    46355.0
2017-08-03 03:00:00    43748.0
2017-08-03 04:00:00    42281.0
Name: ads, dtype: float64

Manchas solares

In [242]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
serie_sunspots = dataframe_sunspots.squeeze('columns')
serie_sunspots = serie_sunspots.resample('ME').mean() # Para los meses
serie_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Funciones

### Gridding

In [243]:
def gridding(lista, grid):
    for serie in lista:
        grid_result = grid.fit(serie.values.reshape(-1,1), serie.values.reshape(-1,1).ravel())
        print(serie.name+" -> Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

### RandomForest

In [244]:
def imputar_rf(serie, lista_entrenamiento, lista_test, n, lista):
    dataframe_rf = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    model = RandomForestRegressor(n_estimators=n)
    lista_series = []
    for i in range(0, len(lista_entrenamiento)):
         
        # Imputamos con la mediana
        model.fit(lista_entrenamiento[i].values.reshape(-1,1), lista_entrenamiento[i].values.reshape(-1,1).ravel())
        serie_rf = model.predict(lista_test[i].values.reshape(-1,1))
        if (i==0):
            print(serie_rf)
        # Calculamos la raiz del error cuadrático medio con las series imputadas con la mediana
        #rmse_rf = auxi.RMSE(serie, lista[i], serie_rf)
        # Calculamos el error medio absoluto con las series imputadas con la mediana
        #mae_rf = auxi.MAE(serie, lista[i], serie_rf)
        # Calculamos el error medio absoluto porcentual con las series imputadas con la mediana
        #mape_rf = auxi.MAPE(serie, lista[i], serie_rf)
        # Calculamos el bias con las series imputadas con la mediana
        #bias_rf = auxi.BIAS(serie, lista[i], serie_rf)

        nueva_fila = pd.DataFrame({ 'Dataset': [lista_entrenamiento[i].name]})
        dataframe_rf = pd.concat([dataframe_rf, nueva_fila], ignore_index=True)
        lista_series.append(serie_rf)
    return dataframe_rf, lista_series

### Crear lag

In [246]:
def create_lag_features(series, lag=3):
    df = pd.DataFrame(series)
    for i in range(1, lag+1):
        df[f'lag_{i}'] = df[series.name].shift(i)
    #df.dropna(inplace=True)  # Remove rows with NaN values
    return df

## Listar

In [247]:
lista_series_temperatura = auxi.leer_series('..\miss_valued_data_temperatura\\')
lista_series_cardiovascular = auxi.leer_series('..\miss_valued_data_cardiovascular\\')
lista_series_sunspots = auxi.leer_series('..\miss_valued_data_manchas_solares\\')

## Escalar datos

### Temperatura

In [248]:
min_max_scaler_temperatura = MinMaxScaler().fit(serie_temperatura.values.reshape(-1,1))
serie_original_temperatura_escalada = auxi.aplicar_escalado(serie_temperatura, min_max_scaler_temperatura)
series_temperatura_escaladas = auxi.escalar(min_max_scaler_temperatura, lista_series_temperatura)

### Cardiovascular

In [249]:
min_max_scaler_cardiovascular = MinMaxScaler().fit(serie_cardiovascular.values.reshape(-1,1))
serie_original_cardiovascular_escalada = auxi.aplicar_escalado(serie_cardiovascular, min_max_scaler_cardiovascular)
series_cardiovascular_escaladas = auxi.escalar(min_max_scaler_cardiovascular, lista_series_cardiovascular)

### Manchas solares

In [250]:
min_max_scaler_sunspots = MinMaxScaler().fit(serie_sunspots.values.reshape(-1,1))
serie_original_sunspots_escalada = auxi.aplicar_escalado(serie_sunspots, min_max_scaler_sunspots)
series_sunspots_escaladas = auxi.escalar(min_max_scaler_sunspots, lista_series_sunspots)

## Separar train y test

In [251]:
series_temperatura_entrenamiento, series_temperatura_test = auxi.train_test(series_temperatura_escaladas)
serie_temperatura_entrenamiento, serie_temperatura_test = auxi.train_test_individual(serie_original_temperatura_escalada)
series_cardiovascular_entrenamiento, series_cardiovascular_test = auxi.train_test(series_cardiovascular_escaladas)
serie_cardiovascular_entrenamiento, serie_cardiovascular_test = auxi.train_test_individual(serie_original_cardiovascular_escalada)
series_sunspots_entrenamiento, series_sunspots_test = auxi.train_test(series_sunspots_escaladas)
serie_sunspots_entrenamiento, serie_sunspots_test = auxi.train_test_individual(serie_original_sunspots_escalada)

## Establecer valores validación

In [252]:
num_folds = 10
seed = 7
scoring = 'neg_root_mean_squared_error'
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
param_grid = dict(n_estimators=range(100,200,50), min_samples_split=range(2,5), min_samples_leaf=range(1,3))
grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, scoring=scoring, cv=kfold)

## Busqueda de hiper-parámetros

In [336]:
df_temperatura_lag = create_lag_features(series_temperatura_escaladas[0], 50)

In [337]:
df_entrenamiento = df_temperatura_lag[df_temperatura_lag[series_temperatura_escaladas[0].name].isna()==False]
df_test = df_temperatura_lag[df_temperatura_lag[series_temperatura_escaladas[0].name].isna()]

In [338]:
Y_test = serie_original_temperatura_escalada[df_temperatura_lag[df_temperatura_lag[series_temperatura_escaladas[0].name].isna()].index]

In [339]:
Y_test

Date
1981-06-01    0.441065
1981-06-02    0.403042
1981-06-03    0.372624
1981-06-04    0.425856
1981-06-05    0.216730
                ...   
1990-10-02    0.399240
1990-10-03    0.376426
1990-10-04    0.418251
1990-10-05    0.532319
1990-10-06    0.349810
Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, Length: 365, dtype: float64

In [340]:
X_train = df_entrenamiento[['lag_1', 'lag_2', 'lag_3']]
Y_train = df_entrenamiento[series_temperatura_entrenamiento[0].name]
X_test = df_test[['lag_1', 'lag_2', 'lag_3']]
#Y_test = df_test[series_temperatura_entrenamiento[0].name]

In [ ]:
#result_rf_e = cross_val_score(RandomForestRegressor(), X, Y, cv=kfold, scoring=scoring)
#print(result_rf_e.mean(), result_rf_e.std())

-0.10219186118634507 0.004899827351881472


In [341]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)
serie_rf = model.predict(X_test)

In [342]:
serie_rf

array([0.51413181, 0.55144487, 0.55418068, 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.57771863, 0.53690589, 0.57404978,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.45032319,
       0.52479087, 0.49132991, 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 ,
       0.4061313 , 0.4061313 , 0.4061313 , 0.4061313 , 0.40613

In [343]:
suma=0
for i in range(0,len(Y_test)):
    suma += (Y_test.iloc[i] - serie_rf[i])**2
suma = math.sqrt(suma/len(Y_test))

In [344]:
suma

0.1745943410465142

In [ ]:
dataframe_temperatura_imputada, lista_series_temperatura_imputada = imputar_rf(serie_temperatura_test, series_temperatura_entrenamiento, series_temperatura_test, 100, series_temperatura_escaladas)

[0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483
 0.40471483 0.40471483 0.40471483 0.40471483 0.40471483 0.4047

### Temperatura

In [255]:
#gridding(series_temperatura_entrenamiento, grid)

### Cardiovascular

In [256]:
#gridding(series_cardiovascular_entrenamiento, grid)

### Manchas solares

In [257]:
#gridding(series_sunspots_entrenamiento, grid)

## Imputamos mediana

In [258]:
'''
#dataframe_temperatura_imputada, lista_series_temperatura_imputada = imputar_mediana(serie_temperatura, lista_series_temperatura)
dataframe_temperatura_imputada, lista_series_temperatura_imputada = imputar_mediana(serie_original_temperatura_escalada, series_temperatura_escaladas)
#dataframe_cardiovascular_imputada, lista_series_cardiovascular_imputada(serie_cardiovascular, lista_series_cardiovascular)
dataframe_cardiovascular_imputada, lista_series_cardiovascular_imputada = imputar_mediana(serie_original_cardiovascular_escalada, series_cardiovascular_escaladas)
#dataframe_sunspots_imputada, lista_series_sunspots_imputada = imputar_mediana(serie_sunspots, lista_series_sunspots)
dataframe_sunspots_imputada, lista_series_sunspots_imputada = imputar_mediana(serie_original_sunspots_escalada, series_sunspots_escaladas)
dataframe_general_imputada = pd.concat([dataframe_temperatura_imputada, dataframe_cardiovascular_imputada, dataframe_sunspots_imputada], ignore_index=True)
dataframe_general_imputada'
'''

"\n#dataframe_temperatura_imputada, lista_series_temperatura_imputada = imputar_mediana(serie_temperatura, lista_series_temperatura)\ndataframe_temperatura_imputada, lista_series_temperatura_imputada = imputar_mediana(serie_original_temperatura_escalada, series_temperatura_escaladas)\n#dataframe_cardiovascular_imputada, lista_series_cardiovascular_imputada(serie_cardiovascular, lista_series_cardiovascular)\ndataframe_cardiovascular_imputada, lista_series_cardiovascular_imputada = imputar_mediana(serie_original_cardiovascular_escalada, series_cardiovascular_escaladas)\n#dataframe_sunspots_imputada, lista_series_sunspots_imputada = imputar_mediana(serie_sunspots, lista_series_sunspots)\ndataframe_sunspots_imputada, lista_series_sunspots_imputada = imputar_mediana(serie_original_sunspots_escalada, series_sunspots_escaladas)\ndataframe_general_imputada = pd.concat([dataframe_temperatura_imputada, dataframe_cardiovascular_imputada, dataframe_sunspots_imputada], ignore_index=True)\ndataframe

## Desescalamos

In [259]:
'''
lista_series_temperatura_imputada = auxi.desescalar(min_max_scaler_temperatura, lista_series_temperatura_imputada)
lista_series_cardiovascular_imputada = auxi.desescalar(min_max_scaler_cardiovascular, lista_series_cardiovascular_imputada)
lista_series_sunspots_imputada = auxi.desescalar(min_max_scaler_sunspots, lista_series_sunspots_imputada)'
'''

"\nlista_series_temperatura_imputada = auxi.desescalar(min_max_scaler_temperatura, lista_series_temperatura_imputada)\nlista_series_cardiovascular_imputada = auxi.desescalar(min_max_scaler_cardiovascular, lista_series_cardiovascular_imputada)\nlista_series_sunspots_imputada = auxi.desescalar(min_max_scaler_sunspots, lista_series_sunspots_imputada)'\n"

## Graficos

### Temperatura

In [260]:
#auxi.graficar_comparacion(serie_temperatura, lista_series_temperatura_imputada)

### Cardiovascular

In [261]:
#auxi.graficar_comparacion(serie_cardiovascular, lista_series_cardiovascular_imputada)

### Manchas solares

In [262]:
#auxi.graficar_comparacion(serie_sunspots, lista_series_sunspots_imputada)

## Guardar datos

In [263]:
'''
dataframe_general_imputada.to_csv("../median/_dataframe_resultados.csv")
auxi.guardar_series("../median/", lista_series_temperatura_imputada)
auxi.guardar_series("../median/", lista_series_cardiovascular_imputada)
auxi.guardar_series("../median/", lista_series_sunspots_imputada)
'''

'\ndataframe_general_imputada.to_csv("../median/_dataframe_resultados.csv")\nauxi.guardar_series("../median/", lista_series_temperatura_imputada)\nauxi.guardar_series("../median/", lista_series_cardiovascular_imputada)\nauxi.guardar_series("../median/", lista_series_sunspots_imputada)\n'